In [ ]:
# 필요한 라이브러리 불러오기  
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install fake_useragent
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
%matplotlib inline
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install chromedriver-autoinstaller



# -*- coding: UTF-8 -*-
import requests
import pandas as pd 
import csv
import time
import re
import numpy as np
import matplotlib.pyplot as plt
import json
import urllib.request as req
import nltk
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from konlpy.tag import Twitter
from collections import Counter
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from bs4 import BeautifulSoup as bs 
import chromedriver_autoinstaller
import os

from selenium import webdriver as wd
import glob
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random


#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# connect google drive
from google.colab import drive
drive.mount('/content/drive')
driver_path = ('/content/drive')
csv_path = ('/content/drive')
#driver = webdriver.Chrome(driver_path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 15.8 MB/s 
     |████████████████████████████████| 384 kB 87.5 MB/s 
     |████████████████████████████████| 140 kB 104.4 MB/s 
ERROR: Operation cancelled by user
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
0% [1 InRelease gpgv 15

ModuleNotFoundError: ignored

In [ ]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver_path = "/content/drive"
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(5)  # or bigger second

In [ ]:
# 검색 키워드 설정: 키워드 내 띄어쓰기는 URL에서 '+'로 표시되기 때문에 이에 맞게 변환
SEARCH_KEYWORD = ''.replace(' ', '+')

In [ ]:
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(3)
# 무한 스크롤 함수 실행
scroll()

스크롤 완료


In [ ]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [ ]:
# 콘텐츠 모든 정보
content_total = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
# 콘텐츠 제목만 추출
content_total_title = list(map(lambda data: data.get_text().replace("\n", ""), content_total))
# 콘텐츠 링크만 추출
content_total_link = list(map(lambda data: "https://youtube.com" + data["href"], content_total))

#--------조회수 & 업로드 날짜 추출(Updated at 2022-10-11)--------#
content_record_src = soup_source.find_all(class_ = 'style-scope ytd-video-meta-block')
content_view_cnt = [content_record_src[i].get_text().replace('조회수 ', '') for i in range(5, len(content_record_src), 10)]
content_upload_date = [content_record_src[i].get_text() for i in range(6, len(content_record_src), 10)]
#---------------------------------------------------------#

# 딕셔너리 포맷팅
content_total_dict = {'title'       : content_total_title, 
                      'link'        : content_total_link, 
                      'view'        : content_view_cnt,
                      'upload_date' : content_upload_date
                      }

In [ ]:
content_total_dict = {'title'       : content_total_title, 
                      'link'        : content_total_link}

In [ ]:
df = pd.DataFrame(content_total_dict)
df

,title,link
0,Introducing iPhone 14 Pro | Apple,https://youtube.com/watch?v=FT3ODSg1GFE
1,iPhone 14 Review: Repackaging 101!,https://youtube.com/watch?v=v94jRN2FhGo
2,iPhone 14 Pro Max - Az egyik leg....abb telefon!,https://youtube.com/watch?v=82N9B1YnrX0
3,"Review: iPhone 14 geen aanrader, 14 Pro wel",https://youtube.com/watch?v=nDrEgbiLz4U
4,iPHONE 14 PLUS UNBOXING + SETUP,https://youtube.com/watch?v=hAXRVXvs5yk
...,...,...
558,深圳华强北 iPhone 14 Pro 跌破5800元，外版双卡下滑1500元，性价比对比入手分析,https://youtube.com/watch?v=ckFsDrC1fpY
559,iPhone 14 Pro Space Black Unboxing + Setup | H...,https://youtube.com/watch?v=zYRYEhgGX68
560,#5 iPhone Accessories (Ultra Budget) 2022 😻😻😻 ...,https://youtube.com/watch?v=lUrfsQMBngk
561,How To Soft Reset iPhone 14 Pro (Step By Step),https://youtube.com/watch?v=EUadUV-1-u0


In [ ]:
df.to_csv("content_total.csv", encoding='utf-8-sig')